In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from collections import Counter, OrderedDict
import scipy.sparse as sp
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import brier_score_loss

In [2]:
conn_string = 'mysql://{user}:{password}@{host}:{port}/'.format(
    user='root', password='dwdstudent2015', 
    host = '34.225.180.235', port=3306)
engine = create_engine(conn_string, encoding='utf8')

engine.execute('USE ml_football')

In [91]:
# all seasons for England
df = pd.read_sql('select Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, HTHG, HTAG, HST, AST, `AS`, `HS` from England', con = engine)

year = []
month = []
results = []
for i in range(df.shape[0]):
    datetime = df['Date'][i].to_pydatetime()
    year.append(datetime.year)
    month.append(datetime.month)
    if df['FTR'][i] == 'D':
        results.append(0)
    elif df['FTR'][i] == 'A':
        results.append(1)
    else:
        results.append(2)
#     df['Date'][i].to_string()
df['Date'] = df['Date'].dt.strftime('%d/%m/%y')
df['results'] = results
df['year'] = year
df['month'] = month
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HST,AST,AS,HS,results,year,month
0,18/08/12,Arsenal,Sunderland,0.0,0.0,D,0.0,0.0,4.0,2.0,3.0,14.0,0,2012,8
1,18/08/12,Fulham,Norwich,5.0,0.0,H,2.0,0.0,9.0,2.0,4.0,11.0,2,2012,8
2,18/08/12,Newcastle,Tottenham,2.0,1.0,H,0.0,0.0,4.0,6.0,12.0,6.0,2,2012,8
3,18/08/12,QPR,Swansea,0.0,5.0,A,0.0,1.0,11.0,8.0,12.0,20.0,1,2012,8
4,18/08/12,Reading,Stoke,1.0,1.0,D,0.0,1.0,3.0,3.0,6.0,9.0,0,2012,8


In [92]:
# season 2012 for England
df1 = df.loc[df['year'] == 2012].loc[df['month'] > 5]
df2 = df.loc[df['year'] == 2013].loc[df['month'] < 6]
df_eng12 = pd.concat([df1,df2])
df_eng12.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HST,AST,AS,HS,results,year,month
0,18/08/12,Arsenal,Sunderland,0.0,0.0,D,0.0,0.0,4.0,2.0,3.0,14.0,0,2012,8
1,18/08/12,Fulham,Norwich,5.0,0.0,H,2.0,0.0,9.0,2.0,4.0,11.0,2,2012,8
2,18/08/12,Newcastle,Tottenham,2.0,1.0,H,0.0,0.0,4.0,6.0,12.0,6.0,2,2012,8
3,18/08/12,QPR,Swansea,0.0,5.0,A,0.0,1.0,11.0,8.0,12.0,20.0,1,2012,8
4,18/08/12,Reading,Stoke,1.0,1.0,D,0.0,1.0,3.0,3.0,6.0,9.0,0,2012,8


In [93]:
def dataTransformer(data,features):
    
    """
    param:
        @original_dataset: (string) name of the csv file
        @features: (list) the features need to be extracted from original dataset
    """
    
#     data = pd.DataFrame.from_csv(original_dataset)
    team_lst = np.unique(data['HomeTeam'].tolist() + data['AwayTeam'].tolist())
    team_store = {team:0 for team in team_lst} 
    
    for team in team_lst:
        home_data = data[data['HomeTeam']==team]
        col_name = home_data.columns.tolist()  
        col_name.insert(col_name.index('AwayTeam')+1,'Home_Indicator')  
        home_data = home_data.reindex(columns=col_name)  
        home_data['Home_Indicator'] = 1
        home_data = home_data.rename(columns={'HomeTeam': 'Team', 'AwayTeam': 'Opponent'})
        
        away_data = data[data['AwayTeam']==team]
        col_name_2 = away_data.columns.tolist()  
        col_name_2.insert(col_name_2.index('AwayTeam')+1,'Home_Indicator')  
        away_data = away_data.reindex(columns=col_name_2)  
        away_data['Home_Indicator'] = 0
        away_data = away_data.rename(columns={'AwayTeam': 'Team', 'HomeTeam': 'Opponent'})
        
        concat_data = pd.concat([home_data, away_data])
        team_data = concat_data[features]
        
#         rename_dict = {'FTHG':'Full_Home_Goals','FTAG':'Full_Away_Goals','FTR':'Full_Results','HTHG':'Half_Home_Goals','HTAG':'Half_Away_Goals',
#                        'HST':'Home_Shots_on_Target','AST':'Away_Shots_on_Target','AS':'Away_Shots','HS':'Home_Shots'}
        rename_dict = {'FTR':'Full_Results'}
        team_data = team_data.rename(columns=rename_dict)
        
        team_data['Date'] = [datetime.strptime(x, '%d/%m/%y') for x in team_data['Date']]
        team_data = team_data.sort_values(by='Date')
        
        #compute the standing until the last game
        col_name_3 = team_data.columns.tolist()  
        col_name_3.insert(col_name_3.index('Full_Results')+1,'Standing')  
        team_data = team_data.reindex(columns=col_name_3)

        tmp = []      
        for pair in zip(team_data['Home_Indicator'], team_data['Full_Results']):
            if (pair[0]==0 and pair[1]=='A') or (pair[0]==1 and pair[1]=='H'):
                tmp.append(3)
            elif (pair[0]==1 and pair[1]=='A') or (pair[0]==0 and pair[1]=='H'):
                tmp.append(0)
            else:
                tmp.append(1)

        standing_lst = []
        for i in range(len(tmp)+2):
            if i > 1:
                standing_lst.append(np.sum(tmp[:i-1]))

        team_data['Standing'] = standing_lst
        
        #################################################################################
        # goals, shots, shots_on_target last 2-5 games
        for k in [5,4,3,2]:
            col_name_4 = team_data.columns.tolist()  
            col_name_4.insert(col_name_4.index('FTHG')+1,'FTHG_last'+str(k)+'_avg')
            col_name_4.insert(col_name_4.index('FTAG')+1,'FTAG_last'+str(k)+'_avg')
            col_name_4.insert(col_name_4.index('HTHG')+1,'HTHG_last'+str(k)+'_avg')
            col_name_4.insert(col_name_4.index('HTAG')+1,'HTAG_last'+str(k)+'_avg')
            col_name_4.insert(col_name_4.index('HST')+1,'HST_last'+str(k)+'_avg')
            col_name_4.insert(col_name_4.index('AST')+1,'AST_last'+str(k)+'_avg')
            team_data = team_data.reindex(columns=col_name_4)
            fthg = []
            ftag = []
            hthg = []
            htag = []
            hst = []
            ast = []
            for i in range(len(tmp)):
                if i - k + 1 < 0:
                    fthg.append(np.sum(team_data['FTHG'][0:i + 1])/k)
                    ftag.append(np.sum(team_data['FTAG'][0:i + 1])/k)
                    hthg.append(np.sum(team_data['HTHG'][0:i + 1])/k)
                    htag.append(np.sum(team_data['HTAG'][0:i + 1])/k)
                    hst.append(np.sum(team_data['HST'][0:i + 1])/k)
                    ast.append(np.sum(team_data['AST'][0:i + 1])/k)
                    
                fthg.append(np.sum(team_data['FTHG'][i - k + 1:i + 1])/k)
                ftag.append(np.sum(team_data['FTAG'][i - k + 1:i + 1])/k)
                hthg.append(np.sum(team_data['HTHG'][i - k + 1:i + 1])/k)
                htag.append(np.sum(team_data['HTAG'][i - k + 1:i + 1])/k)
                hst.append(np.sum(team_data['HST'][i - k + 1:i + 1])/k)
                ast.append(np.sum(team_data['AST'][i - k + 1:i + 1])/k)
            team_data['FTHG_last'+str(k)+'_avg']= fthg
            team_data['FTAG_last'+str(k)+'_avg']= ftag
            team_data['HTHG_last'+str(k)+'_avg']= hthg
            team_data['HTAG_last'+str(k)+'_avg']= htag
            team_data['HST_last'+str(k)+'_avg']= hst
            team_data['AST_last'+str(k)+'_avg']= ast
        #################################################################################
        
        #compute the winning probability
        col_name_4 = team_data.columns.tolist()  
        col_name_4.insert(col_name_4.index('Standing')+1,'Winning_Probability')
        team_data = team_data.reindex(columns=col_name_4)
        
        col_name_4 = team_data.columns.tolist()
        col_name_4.insert(col_name_4.index('Winning_Probability')+1,'Winning_Probability_last5')
        team_data = team_data.reindex(columns=col_name_4)
        
        tmp_1 = []
        for pair in zip(team_data['Home_Indicator'], team_data['Full_Results']):
            if (pair[0]==0 and pair[1]=='A') or (pair[0]==1 and pair[1]=='H'):  #win
                tmp_1.append(1)
            elif (pair[0]==1 and pair[1]=='A') or (pair[0]==0 and pair[1]=='H'): #lose
                tmp_1.append(0)
            else: #draw
                tmp_1.append(0)
        
        win_lst = []
        win_last5 = [0,0,0,0,0]
        for i in range(len(tmp_1)):
            win_lst.append(np.sum(tmp_1[:i + 1])/(i + 1))
        for i in range(len(tmp_1)):
            if i > 4:
                win_last5.append(np.sum(tmp_1[i-4:i + 1])/5)
        team_data['Winning_Probability']= win_lst
                                 
        
        team_data['Winning_Probability_last5'] = win_last5
        
        #################################################################################
        
        #compute the winning probability of being home/away team
        home_wprob_lst = []
        away_wprob_lst = []
        
        for i in range(len(team_data)):
            tmp_data = team_data[:i+1]
            tmp_df = tmp_data[tmp_data['Home_Indicator']==1]
            if len(tmp_df)==0:
                home_wprob_lst.append(0)
            else:
                home_wprob = (len(tmp_df[tmp_df['Full_Results']=='H']))/len(tmp_df)
                home_wprob_lst.append(home_wprob)

            tmp_df1 = tmp_data[tmp_data['Home_Indicator']==0]
            if len(tmp_df1)==0:
                away_wprob_lst.append(0)
            else:
                away_wprob = (len(tmp_df1[tmp_df1['Full_Results']=='A']))/len(tmp_df1)
                away_wprob_lst.append(away_wprob)
        
        col_name = team_data.columns.tolist() 
        col_name.insert(col_name.index('Winning_Probability_last5')+1,'Home_Win_Prob')
        team_data = team_data.reindex(columns=col_name)
        team_data['Home_Win_Prob'] = home_wprob_lst

        col_name = team_data.columns.tolist() 
        col_name.insert(col_name.index('Home_Win_Prob')+1,'Away_Win_Prob')
        team_data = team_data.reindex(columns=col_name)
        team_data['Away_Win_Prob'] = away_wprob_lst
        
        rename_dict = {'FTHG':'Full_Home_Goals','FTAG':'Full_Away_Goals','HTHG':'Half_Home_Goals','HTAG':'Half_Away_Goals',
                       'HST':'Home_Shots_on_Target','AST':'Away_Shots_on_Target','AS':'Away_Shots','HS':'Home_Shots'}
        team_data = team_data.rename(columns=rename_dict)
        
        team_store[team] = team_data
        
        
        
    return team_store

In [96]:
features_lst = ['Date','Team','Opponent','Home_Indicator','FTHG', 'FTAG', 'FTR','HTHG','HTAG','HST','AST','AS','HS']
dict_eng12 = dataTransformer(df_eng12, features_lst)
dict_eng12['QPR']

,Date,Team,Opponent,Home_Indicator,Full_Home_Goals,FTHG_last2_avg,FTHG_last3_avg,FTHG_last4_avg,FTHG_last5_avg,Full_Away_Goals,...,HST_last3_avg,HST_last4_avg,HST_last5_avg,Away_Shots_on_Target,AST_last2_avg,AST_last3_avg,AST_last4_avg,AST_last5_avg,Away_Shots,Home_Shots
3,2012-08-18,QPR,Swansea,1,0.0,0.0,0.000000,0.00,0.0,5.0,...,0.000000,0.00,0.0,8.0,0.0,0.000000,0.00,0.0,12.0,20.0
14,2012-08-25,QPR,Norwich,0,1.0,0.5,0.000000,0.00,0.0,1.0,...,0.000000,0.00,0.0,4.0,6.0,0.000000,0.00,0.0,6.0,13.0
20,2012-09-01,QPR,Man City,0,3.0,2.0,1.333333,0.00,0.0,1.0,...,9.000000,0.00,0.0,5.0,4.5,5.666667,0.00,0.0,9.0,19.0
34,2012-09-15,QPR,Chelsea,1,0.0,1.5,1.333333,1.00,0.0,0.0,...,7.333333,8.25,0.0,9.0,7.0,6.000000,6.50,0.0,13.0,10.0
48,2012-09-23,QPR,Tottenham,0,2.0,1.0,1.666667,1.50,1.2,1.0,...,10.666667,9.00,9.4,6.0,7.5,6.666667,6.00,6.4,9.0,18.0
58,2012-10-01,QPR,West Ham,1,1.0,1.5,1.000000,1.50,1.4,2.0,...,9.666667,10.25,9.0,10.0,8.0,8.333333,7.50,6.8,17.0,13.0
62,2012-10-06,QPR,West Brom,0,3.0,2.0,2.000000,1.50,1.8,2.0,...,10.333333,9.25,9.8,6.0,8.0,7.333333,7.75,7.2,13.0,13.0
77,2012-10-21,QPR,Everton,1,1.0,2.0,1.666667,1.75,1.4,1.0,...,7.666667,9.25,8.6,4.0,5.0,6.666667,6.50,7.0,10.0,14.0
79,2012-10-27,QPR,Arsenal,0,1.0,1.0,1.666667,1.50,1.6,0.0,...,8.333333,8.50,9.6,3.0,3.5,4.333333,5.75,5.8,4.0,21.0
97,2012-11-04,QPR,Reading,1,1.0,1.0,1.000000,1.50,1.4,1.0,...,9.666667,9.25,9.2,8.0,5.5,5.000000,5.25,6.2,18.0,23.0


In [107]:
df = pd.DataFrame()
i = 0
for k in dict_eng12.keys():
    i += 1
    tmp = dict_eng12[k]
    a = tmp.iloc[:, 1:4].reset_index(drop = True)
    init_row = pd.DataFrame(columns=tmp.columns[4:]).reset_index(drop=True)
    init_row.loc[0] = [0]*len(tmp.columns[4:])
    b = pd.concat([init_row,tmp.iloc[:-1,4:]]).reset_index(drop=True)
    new = pd.concat([a, b],axis = 1)
    df = pd.concat([df, new], axis = 0)    
df.reset_index(drop = True)
df.to_csv('eng12.csv')